# Sentiment Anlaysis using Transfer Learning -Bert

# Step 1: Install And Import Python Libraries

In step 1, we will install and import python libraries.

Firstly, let's import `transformers` and `datasets`.

In [1]:
# Install libraries
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


After installing the python packages, we will import the python libraries.
* `pandas` and `numpy` are imported for data processing.
* `train_test_split` is imported from `sklearn` to split dataset.
* `tensorflow` and `transformers` are imported for modeling.
* `Dataset` is imported for the Hugging Face dataset format.
* The `accuracy_score` is imported for model performance evaluation.

In [2]:
# Data processing
import pandas as pd
import numpy as np

# Train test split
from sklearn.model_selection import train_test_split

# Modeling
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Hugging Face Dataset
from datasets import Dataset

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

# Step 2: Download And Read Data

Now let's read the data into a `pandas` dataframe and see what the dataset looks like.

The dataset has two columns. One column contains the reviews and the other column contains the sentiment label for the review.

In [3]:
# Read in data
sst_train = pd.read_csv('/content/sample_data/SST_Train_Data.csv')
sst_test = pd.read_csv('/content/sample_data/SST_Test_Data.csv')

# Take a look at the data
sst_train.head()

reviews     label  label_Value
0  the rock is destined to be the 21st century s ...  positive            2
1  the gorgeously elaborate continuation of  the ...  positive            2
2  singercomposer bryan adams contributes a slew ...  positive            2
3  you  would think by now america would have had...   neutral            1
4                yet the act is still charming here   positive            2

`.info` helps us to get information about the dataset.

From the output, we can see that this data set has 8117 records and no missing data. The `reviews` column is the `object` type and the `label_Value` column is the `int64` type.

In [4]:
# Get the dataset information
sst_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8117 entries, 0 to 8116
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviews      8117 non-null   object
 1   label        8117 non-null   object
 2   label_Value  8117 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 190.4+ KB


In [5]:
# Check the label distribution
sst_train['label_Value'].value_counts()

2    3446
0    3122
1    1549
Name: label_Value, dtype: int64

# Step 3: Tokenize Text

In step 3, we will tokenize the review text using a tokenizer.

A tokenizer converts text into numbers to use as the input of the NLP (Natural Language Processing) models. Each number represents a token, which can be a word, part of a word, punctuation, or special tokens.
* `AutoTokenizer.from_pretrained("bert-base-cased")` downloads vocabulary from the pretrained `bert-base-cased` model.
* `return_tensors="np"` indicates that the return format is numpy array. Besides `np`, `return_tensors` can take the value of `tf` or `pt`, where `tf` returns Tensorflow `tf.constant` object and `pt` returns PyTorch `torch.tensor` object. If not set, it returns a list of python integers.
* `padding` means adding zeros to shorter reviews in the dataset. The `padding` argument controls how `padding` is implemented.  
 * `padding=True` is the same as `padding='longest'`. It checks the longest sequence in the batch and pads zeros to that length. There is no padding if only one text document is provided.
 * `padding='max_length'` pads to `max_length` if it is specified, otherwise, it pads to the maximum acceptable input length for the model.
 * `padding=False` is the same as `padding='do_not_pad'`. It is the default, indicating that no padding is applied, so it can output a batch with sequences of different lengths.

The labels for the reviews are converted to one-dimensional numpy arrays.

In [6]:
## Tokenize Reviews
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

train_data = sst_train["reviews"].tolist()
test_data = sst_test["reviews"].tolist()
# Tokenize the reviews
tokenized_data_train = tokenizer(train_data, return_tensors="np", padding=True)
tokenized_data_test = tokenizer(test_data, return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = sst_train["label_Value"].tolist()
labels_test = sst_test["label_Value"].tolist()
labels_train = tf.convert_to_tensor(labels_train, dtype=tf.int32)
labels_test = tf.convert_to_tensor(labels_test, dtype=tf.int32)

# Tokenized ids
print(tokenized_data_train["input_ids"][0])

[  101  1103  2067  1110 17348  1106  1129  1103  6880  1432   188  1207
 14255  1389  1105  1115  1119   188  1280  1106  1294   170 24194  1256
  3407  1190   170 11791  5253   188  1732  7200 10947 12606  2895   179
  7766  1665 15554  1181  3498  6961  3263  1137   188  1566  7912 14516
  6997   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0]


After printing out the tokenized IDs for the first review, we can see that the tokenized words are converted into integers, and the sentence is padded with zeros at the end of the review.

There are two special tokens in the token IDs, `101` at the beginning of the sentence and `102` at the end of the sentence. The BERT tokenizer uses `101` to encode the special token [CLS] and `102` to encode the special token [SEP], but the other models may use other special tokens.

# Step 4: Compile Transfer Learning Model for Sentiment Analysis

In step 4, we will build a customized transfer learning model for sentiment analysis.

* `TFAutoModelForSequenceClassification` loads the BERT model without the sequence classification head.
* The method `from_pretrained()` loads the weights from the pretrained model into the new model, so the weights in the new model are not randomly initialized. Note that the new weights for the new sequence classification head are going to be randomly initialized.
* `bert-base-cased` is the name of the pretrained model. We can change it to a different model based on the nature of the project.
* `num_labels` indicates the number of classes. Our dataset has three classes, positive,neutral and negative, so `num_labels=3`.

In [7]:
# Load model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


After loading the pretrained model, we will compile the model.
* `SparseCategoricalCrossentropy` is used as the loss function, but the Hugging Face documentation mentioned that Hugging Face models automatically choose a loss that is appropriate for their task and model architecture if the loss is not explicitly specified.
* `from_logits=True` informs the loss function that the output values are logits before applying softmax, so the values do not represent probabilities.
* We are using Adam as the optimizer and the number `5e-6` is the learning rate. A smaller learning rate corresponds to a more stable weights value update and a slower training process.
* `accuracy` is used as the metrics because we have a balanced dataset.

In [8]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=Adam(5e-6), loss=loss, metrics=['accuracy'])

# Step 5: Train Transfer Learning Model for Sentiment Analysis

In step 5, we will talk about how to fit a transfer learning model using Tensorflow Keras.

When fitting the model, we convert the tokenized dataset into a dictionary for Keras. `batch_size=4` means that four reviews are processed for each weights and bias update. `epochs=15` means that the model fitting process will go through the training dataset 15 times.

We can see that the accuracy is above 98 percent in 15 epochs.

1.   List item
2.   List item



In [11]:
# Fit the model
model.fit(dict(tokenized_data_train),
           labels_train,
           validation_data=(dict(tokenized_data_test), labels_test),
           batch_size=4,
           epochs=15)

All the weights will be updated by default for the transfer learning model.

* If we would like to keep the pretrained model weights as is and only update the weights and bias of the output layer, we can use `model.layers[0].trainable = False` to freeze the weights of the BERT model.
* If we would like to keep the weights of some layers and update others, we can use `model.bert.encoder.layer[i].trainable = False` to freeze the weights of the corresponding layers.
* In general, if the dataset for the transfer learning model is large, it is suggested to update all weights, and if the dataset for the transfer learning model is small, it is suggested to freeze the pretrained model weights. But we can always compare the model performance by adding the tunable pretrained model layers one by one.

# Step 6: Sentiment Analysis Transfer Learning Model Prediction and Evaluation

In step 6, we will see model prediction and evaluation for sentiment analysis transfer learning.

Passing the tokenized text to the `.predict` method, we get the predictions for the customized transfer learning sentiment model. `logits` is the last layer of the neural network before softmax is applied.

We can see that the prediction has two columns. The first column is the predicted logit for label 0 and the second column is the predicted logit for label 1. logit values do not sum up to 1.

In [ ]:
# Predictions
y_test_predict = model.predict(dict(tokenized_data_test))['logits']

# First 5 predictions
y_test_predict[:5]

67/67 [==============================] - 12s 138ms/step


array([[-2.1745813,  4.9053063, -3.2688015],
       [-4.33564  , -1.0755202,  5.0015435],
       [-5.835117 ,  1.378774 ,  3.7665915],
       [-3.865402 , -1.8417912,  5.4970236],
       [-2.2431595, -2.1613474,  4.6802683]], dtype=float32)

To get the predicted probabilities, we need to apply softmax on the predicted logit values.

After applying softmax, we can see that the predicted probability for each review sums up to 1.

In [ ]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_predict)

# First 5 predicted probabilities
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[8.4092299e-04, 9.9887758e-01, 2.8154152e-04],
       [8.7877823e-05, 2.2894500e-03, 9.9762267e-01],
       [6.1922598e-05, 8.4101215e-02, 9.1583687e-01],
       [8.5828338e-05, 6.4934249e-04, 9.9926478e-01],
       [9.8243309e-04, 1.0661873e-03, 9.9795145e-01]], dtype=float32)>

To get the predicted labels, `argmax` is used to return the index of the maximum probability for each review, which corresponds to the labels of zeros and ones.

In [ ]:
# Predicted label
y_test_class_preds = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted labels
y_test_class_preds[:10]

array([1, 2, 2, 2, 2, 2, 2, 1, 2, 2])

`accuracy_score` is used to evaluate the model performance. We can see that the customized sentiment analysis model with transfer learning gives us 71% accuracy, meaning that the predictions are correct 71% of the time.

In [ ]:
# Accuracy
accuracy_score(y_test_class_preds, labels_test)

0.715764705882353

# Step 7: Save and Load Model

In step 7, we will talk about how to save the model and reload it for prediction.

`tokenizer.save_pretrained` saves the tokenizer information to the drive and `model.save_pretrained` saves the model to the drive.

In [ ]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_tensorflow/')

# Save model
model.save_pretrained('./sentiment_transfer_learning_tensorflow/')

We can load the saved tokenizer later using `AutoTokenizer.from_pretrained()` and load the saved model using `TFAutoModelForSequenceClassification.from_pretrained()`.

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_tensorflow/")

# Load model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_tensorflow/')

Some layers from the model checkpoint at ./sentiment_transfer_learning_tensorflow/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./sentiment_transfer_learning_tensorflow/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


To verify that the customized transfer learning model is loaded correctly, the loaded model is used to make predictions on the testing dataset. We can see that the prediction results are exactly the same as the fine-tuned model, confirming that the model is loaded correctly.

In [ ]:
# Predict logit using the loaded model
y_test_predict = loaded_model.predict(dict(tokenized_data_test))['logits']

# Take a look at the first 5 predictions
y_test_predict[:5]

67/67 [==============================] - 12s 134ms/step


array([[-2.1745813,  4.9053063, -3.2688015],
       [-4.33564  , -1.0755202,  5.0015435],
       [-5.835117 ,  1.378774 ,  3.7665915],
       [-3.865402 , -1.8417912,  5.4970236],
       [-2.2431595, -2.1613474,  4.6802683]], dtype=float32)